In [79]:
import numpy as np
import random

In [80]:
def softmax(x):
    e_x = np.exp(x - np.max(x))
    return e_x / e_x.sum(axis=0)

In [81]:
def get_initial_loss(vocab_size, seq_length):
    return -np.log(1.0/vocab_size)*seq_length

In [82]:
def smooth(loss, cur_loss):
    return loss * 0.999 + cur_loss * 0.001

In [83]:
def print_sample(sample_ix, ix_to_char):
    txt = ''.join(ix_to_char[ix] for ix in sample_ix)
    txt = txt[0].upper() + txt[1:]  # capitalize first character 
    print ('%s' % (txt, ), end='')

In [84]:
def initialize_parameters(n_a, n_x, n_y):
#     np.random.seed(1)
    Wax = np.random.randn(n_a, n_x)*0.01 
    Waa = np.random.randn(n_a, n_a)*0.01 
    Wya = np.random.randn(n_y, n_a)*0.01 
    ba = np.zeros((n_a, 1)) 
    by = np.zeros((n_y, 1)) 
    
    parameters = {"Wax": Wax, "Waa": Waa, "Wya": Wya, "ba": ba,"by": by}
    
    return parameters

In [85]:
def update_parameters(parameters, gradients, lr):

    parameters['Wax'] += -lr * gradients['dWax']
    parameters['Waa'] += -lr * gradients['dWaa']
    parameters['Wya'] += -lr * gradients['dWya']
    parameters['ba']  += -lr * gradients['dba']
    parameters['by']  += -lr * gradients['dby']
    return parameters

In [86]:
def rnn_step_forward(parameters, a_prev, x):
    
    Waa, Wax, Wya, by, ba = parameters['Waa'], parameters['Wax'], parameters['Wya'], parameters['by'], parameters['ba']
    a_next = np.tanh(np.dot(Wax, x) + np.dot(Waa, a_prev) + ba) 
    p_t = softmax(np.dot(Wya, a_next) + by)  
    
    return a_next, p_t

In [87]:
def rnn_forward(X, Y, a0, parameters, vocab_size = 27):
    
    x, a, y_hat = {}, {}, {}
    
    a[-1] = np.copy(a0)
    
    loss = 0
    
    for t in range(len(X)):
        
        x[t] = np.zeros((vocab_size,1)) 
        if (X[t] != None):
            x[t][X[t]] = 1
        
        a[t], y_hat[t] = rnn_step_forward(parameters, a[t-1], x[t])
        
        loss -= np.log(y_hat[t][Y[t],0])
        
    cache = (y_hat, a, x)
        
    return loss, cache


In [88]:
def rnn_step_backward(dy, gradients, parameters, x, a, a_prev):
    
    gradients['dWya'] += np.dot(dy, a.T)
    gradients['dby'] += dy
    da = np.dot(parameters['Wya'].T, dy) + gradients['da_next'] 
    dz = (1 - a * a) * da 
    gradients['dba'] += dz
    gradients['dWax'] += np.dot(dz, x.T)
    gradients['dWaa'] += np.dot(dz, a_prev.T)
    gradients['da_next'] = np.dot(parameters['Waa'].T, dz)
    return gradients

In [89]:
def rnn_backward(X, Y, parameters, cache):
    gradients = {}
    
    (y_hat, a, x) = cache
    Waa, Wax, Wya, by, ba = parameters['Waa'], parameters['Wax'], parameters['Wya'], parameters['by'], parameters['ba']
    
    gradients['dWax'], gradients['dWaa'], gradients['dWya'] = np.zeros_like(Wax), np.zeros_like(Waa), np.zeros_like(Wya)
    gradients['dba'], gradients['dby'] = np.zeros_like(ba), np.zeros_like(by)
    gradients['da_next'] = np.zeros_like(a[0])
    
    for t in reversed(range(len(X))):
        dy = np.copy(y_hat[t])
        dy[Y[t]] -= 1
        gradients = rnn_step_backward(dy, gradients, parameters, x[t], a[t], a[t-1])
    
    return gradients, a


In [90]:
file_name='names.txt'

In [91]:
data = open(file_name, 'r').read()
data= data.lower()
print(type(data))
# print(data.shape, data)
chars = list(set(data))
data_size, vocab_size = len(data), len(chars)
print('There are %d total characters and %d unique characters in your data.' % (data_size, vocab_size))

<class 'str'>
There are 45594 total characters and 27 unique characters in your data.


In [92]:
print(chars)
char_to_ix = { ch:i for i,ch in enumerate(sorted(chars)) }
ix_to_char = { i:ch for i,ch in enumerate(sorted(chars)) }
print(ix_to_char, char_to_ix)
# print(data)

['w', '\n', 'l', 'b', 'g', 'e', 'r', 'u', 'z', 'o', 's', 'x', 'f', 'k', 'm', 'a', 'p', 'd', 'h', 'c', 'n', 'y', 'q', 't', 'j', 'v', 'i']
{0: '\n', 1: 'a', 2: 'b', 3: 'c', 4: 'd', 5: 'e', 6: 'f', 7: 'g', 8: 'h', 9: 'i', 10: 'j', 11: 'k', 12: 'l', 13: 'm', 14: 'n', 15: 'o', 16: 'p', 17: 'q', 18: 'r', 19: 's', 20: 't', 21: 'u', 22: 'v', 23: 'w', 24: 'x', 25: 'y', 26: 'z'} {'w': 23, 'a': 1, '\n': 0, 'l': 12, 'b': 2, 'p': 16, 'g': 7, 'c': 3, 'h': 8, 'e': 5, 'd': 4, 'r': 18, 'n': 14, 'y': 25, 'u': 21, 'q': 17, 't': 20, 'z': 26, 'j': 10, 'o': 15, 'v': 22, 's': 19, 'x': 24, 'f': 6, 'k': 11, 'i': 9, 'm': 13}


In [93]:
def clip(gradients, maxValue):
    
    dWaa, dWax, dWya, dba, dby = gradients['dWaa'], gradients['dWax'], gradients['dWya'], gradients['dba'], gradients['dby']
   
    for gradient in [dWax, dWaa, dWya, dba, dby]:
        np.clip(gradient, -maxValue, maxValue, out = gradient)
    
    gradients = {"dWaa": dWaa, "dWax": dWax, "dWya": dWya, "dba": dba, "dby": dby}
    
    return gradients

In [94]:
# def sample(parameters, char_to_ix, seed):
def sample(parameters, char_to_ix):
    
    
    Waa, Wax, Wya, by, ba = parameters['Waa'], parameters['Wax'], parameters['Wya'], parameters['by'], parameters['ba']
    vocab_size = by.shape[0]
    n_a = Waa.shape[1]
    
    x = np.zeros((vocab_size, 1))
    a_prev = np.zeros((n_a, 1))
    
    indices = []
    
    idx = -1 
    
    
    counter = 0
    newline_character = char_to_ix['\n']
    
    while (idx != newline_character and counter != 50):
        
        a = np.tanh(np.dot(Wax, x) + np.dot(Waa, a_prev) + ba)
        z = np.dot(Wya, a) + by
        y = softmax(z)
        
#         np.random.seed(counter+seed) 
        
        idx = np.random.choice(list(range(vocab_size)), p=y.ravel())

        indices.append(idx)
        
        x = np.zeros((vocab_size, 1))
        x[idx] = 1
        
        a_prev = a
        
#         seed += 1
        counter +=1
        

    if (counter == 50):
        indices.append(char_to_ix['\n'])
    
    return indices

In [95]:
def optimize(X, Y, a_prev, parameters, learning_rate = 0.01):
    
    
    
    loss, cache = rnn_forward(X, Y, a_prev, parameters)
    
    gradients, a = rnn_backward(X, Y, parameters, cache)
    
    gradients = clip(gradients, 5)
    
    parameters = update_parameters(parameters, gradients, learning_rate)
    
    
    return loss, gradients, a[len(X)-1]

In [96]:

def model(data, ix_to_char, char_to_ix, num_iterations = 35000, n_a = 50, dino_names = 7, vocab_size = 27):
    
    
    n_x, n_y = vocab_size, vocab_size
    
    parameters = initialize_parameters(n_a, n_x, n_y)
    
    loss = get_initial_loss(vocab_size, dino_names)
    
    with open(file_name) as f:
        examples = f.readlines()
    examples = [x.lower().strip() for x in examples]
    
#     np.random.seed(0)
    np.random.shuffle(examples)
    
    a_prev = np.zeros((n_a, 1))
    
    for j in range(num_iterations):
        
        
        index = j % len(examples)
        X = [None] + [char_to_ix[ch] for ch in examples[index]]
        Y = X[1:] + [char_to_ix["\n"]]
        
        curr_loss, gradients, a_prev = optimize(X, Y, a_prev, parameters)
        
        
        loss = smooth(loss, curr_loss)

        if j % 2000 == 0:
            
            print('Iteration: %d, Loss: %f' % (j, loss) + '\n')
            
#             seed = 0
            for name in range(dino_names):
                
#                 sampled_indices = sample(parameters, char_to_ix, seed)
                sampled_indices = sample(parameters, char_to_ix)
                print_sample(sampled_indices, ix_to_char)
                
#                 seed += 1  
      
            print('\n')
        
    return parameters

In [97]:
# print(data)
parameters = model(data, ix_to_char, char_to_ix)

Iteration: 0, Loss: 19.775981

Rkwixnxtesobjgnnerlpypkpuvilxjpttccivdqfqjvvgcabbc
Kceyvbelazuhxjtnzoosktayancrhpzixrscbnj
Nsylo
Fjmxgegsoga
Gtexqtubftxawbxeozodwkdco
Rmgex
Baxhxmjgpjtwivjveayek


Iteration: 2000, Loss: 20.178975

Anq
Aj
Ot
Jam

Ibikikm



Iteration: 4000, Loss: 17.995098

Bhurazin
Sirit
Eep
Amavcherbaminan
Aninjardoh
Dik
Sunveed


Iteration: 6000, Loss: 14.057537

Mramisogudashujilajamajajurantapes
Inu
Gojnimhenhaghushish
Jabandrajulakh
Rakenday
Malesh
Aghak


Iteration: 8000, Loss: 19.967727

Ragashun
Aokam
Sinay
Jinden
Sohaan
Munad
Aepam


Iteration: 10000, Loss: 12.093945

Fuinakh
Mannder
Manur
Muaneek
Eppandendrravit
Muthan
Amesh


Iteration: 12000, Loss: 16.956051

Sulunk
Sont
Mundeer
Ak
Satan
Vukesh
Deepa


Iteration: 14000, Loss: 12.176500

Mintarsjesh
Birujal
Moram
Wayfunsh
Kunish
Ovtukash
Antouk


Iteration: 16000, Loss: 14.263094

Andeet
Rangu
Mohir
Mamman
Ovu
Kudeep
Deepal


Iteration: 18000, Loss: 9.374336

Santhntul
Sandeep
Onit
Vadet
Oripra
Kip
Deepak


I